### Домашня работа №3
1. обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта
2. при обучении моделей обязательно использовать кроссвалидацию
3. вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)
4. сделать выводы о том, какая модель справилась с задачей лучше других
5. (опциональный вопрос) какая метрика (precision_recall_curve или roc_auc_curve) больше подходит в случае сильного дисбаланса классов? (когда объектов одного из классов намного больше чем другого). 

p.s.В вопросе проще разобраться, если вспомнить оси на графике roc auc curve и рассмотреть такой пример:

Имеется 100000 объектов, из которых только 100 - класс "1" (99900 - класс "0", соответственно). 
Допустим, у нас две модели:

- первая помечает 100 объектов как класс 1, но TP = 90
- вторая помечает 1000 объектов как класс 1, но TP такой же - 90

Какая модель лучше и почему? И что позволяет легче сделать вывод - roc_auc_curve или precision_recall_curve?

#### 1. обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта
#### 2. при обучении моделей обязательно использовать кроссвалидацию

In [53]:
import numpy as np
import pandas as pd
import itertools
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from scipy.sparse import hstack
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV

%matplotlib inline
sns.set(font_scale=1.5)
sns.set_color_codes("muted")


Нужно предсказать наличие сердечно-сосудистых заболеваний по результатам классического врачебного осмотра. Датасет сформирован из 100.000 реальных клинических анализов, и в нём используются признаки, которые можно разбить на 3 группы:

 

Объективные признаки:

 - Возраст
 - Рост
 - Вес
 - Пол
 

Результаты измерения:

 - Артериальное давление верхнее и нижнее
 - Холестерин
 - Глюкоза
 

Субъективные признаки:

 - Курение
 - Употребление Алкоголя
 - Физическая активность
 

Возраст дан в днях. Значения показателей холестерина и глюкозы представлены одним из трех классов: норма, выше нормы, значительно выше нормы. Значения субъективных признаков — бинарны.

Все показатели даны на момент осмотра.

Таргет - наличие сердечно-сосудистых заболеваний (ССЗ)

In [54]:
df = pd.read_csv('train_case2.csv', ';')
df.info()

c:\Users\60053972\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           70000 non-null  int64  
 1   age          70000 non-null  int64  
 2   gender       70000 non-null  int64  
 3   height       70000 non-null  int64  
 4   weight       70000 non-null  float64
 5   ap_hi        70000 non-null  int64  
 6   ap_lo        70000 non-null  int64  
 7   cholesterol  70000 non-null  int64  
 8   gluc         70000 non-null  int64  
 9   smoke        70000 non-null  int64  
 10  alco         70000 non-null  int64  
 11  active       70000 non-null  int64  
 12  cardio       70000 non-null  int64  
dtypes: float64(1), int64(12)
memory usage: 6.9 MB


Разделим наши данные на тренировочную и тестовую выборки

In [55]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df.drop('cardio', 1), 
                                                    df['cardio'], random_state=0)

C:\Users\60053972\AppData\Local\Temp/ipykernel_18076/3802500363.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X_train, X_test, y_train, y_test = train_test_split(df.drop('cardio', 1),


К полям:
- gender, cholesterol применим OHE-кодирование
- age, height, weight, ap_hi, ap_lo - MinMaxScaler
- gluc, smoke, alco, active - оставим пока как есть

In [56]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]


from sklearn.preprocessing import StandardScaler


continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

Теперь объединим все наши трансформеры с помощью FeatureUnion

In [57]:
from sklearn.pipeline import FeatureUnion

feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)
feature_processing = Pipeline([('feats', feats)])

feature_processing.fit_transform(X_train)

array([[-1.73391771,  0.6873301 ,  0.74843904, ...,  1.        ,
         0.        ,  1.        ],
       [-1.67343538,  0.07758923, -0.29640123, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.13738132,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.17775864,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  1.        ],
       [-0.47190715, -1.38578883,  0.74843904, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.38174619,  0.56538192, -0.08743318, ...,  0.        ,
         0.        ,  1.        ]])

#### LogisticRegression

In [58]:
classifier = Pipeline([
    ('features',feats),
    ('classifier', LogisticRegression(random_state = 42)),
])


#запустим кросс-валидацию
cv_scores = cross_val_score(classifier, X_train, y_train, cv=16, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std = np.std(cv_scores)
print('CV score is {}+-{}'.format(cv_score, cv_score_std))

#обучим пайплайн на всем тренировочном датасете
classifier.fit(X_train, y_train)
y_score = classifier.predict_proba(X_test)[:, 1]

CV score is 0.7867401104915408+-0.00852135511666111


Посчитаем precision/recall/f_score

In [59]:
b=1
precision_LR, recall_LR, thresholds_LR = precision_recall_curve(y_test.values, y_score)
fscore_LR = (1+b**2)*(precision_LR * recall_LR) / (b**2*precision_LR + recall_LR)
# locate the index of the largest f score
ix_LR = np.argmax(fscore_LR)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds_LR[ix_LR], 
                                                                        fscore_LR[ix_LR],
                                                                        precision_LR[ix_LR],
                                                                        recall_LR[ix_LR]))

Best Threshold=0.386937, F-Score=0.730, Precision=0.647, Recall=0.838


In [60]:
# посчитаем ROC AUC
roc_auc_LR = roc_auc_score(y_test.values, y_score)
roc_auc_LR

0.7840347790421852

Подберем параметры с помощью кросс-валидации

In [61]:
classifier = Pipeline([
    ('features', feats),
    ('classifier', LogisticRegression(random_state = 42)),
])

params={'classifier__penalty':["none", "l2"],
        'classifier__solver':["lbfgs", "liblinear"],
        'classifier__max_iter':[100, 200]
        }

Запускаем перебор

In [62]:
grid = GridSearchCV(classifier,
                    param_grid=params,
                    cv=5,
                    refit=False)

search = grid.fit(X_train, y_train)
search.best_params_

c:\Users\60053972\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 40.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\60053972\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\60053972\Anaconda3\lib\site-packages\sklearn\pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "c:\Users\60053972\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, se

{'classifier__max_iter': 100,
 'classifier__penalty': 'none',
 'classifier__solver': 'lbfgs'}

Обучаем модель уже с новыми параметрами

In [66]:
classifier = Pipeline([
    ('features', feats),
    ('classifier', LogisticRegression(max_iter = 100, penalty = 'none', solver = 'lbfgs', random_state = 42)),
])
classifier.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('age',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='age')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('height',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='height')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('weight',
                                                 Pipeline(steps=[('selector',
                                        

In [67]:
y_score = classifier.predict_proba(X_test)[:, 1]
b=1
precision_LR, recall_LR, thresholds_LR = precision_recall_curve(y_test.values, y_score)
fscore_LR = (1+b**2)*(precision_LR * recall_LR) / (b**2*precision_LR + recall_LR)
# locate the index of the largest f score
ix_LR = np.argmax(fscore_LR)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds_LR[ix_LR], 
                                                                        fscore_LR[ix_LR],
                                                                        precision_LR[ix_LR],
                                                                        recall_LR[ix_LR]))

Best Threshold=0.385989, F-Score=0.730, Precision=0.647, Recall=0.838


In [68]:
roc_auc_LR = roc_auc_score(y_test.values, y_score)
roc_auc_LR

0.7842574349248148

#### CatBoostClassifaier

In [69]:
classifier_CB = Pipeline([
    ('features',feats),
    ('classifier', CatBoostClassifier(verbose = False, random_state = 42)),
])

#обучим пайплайн на всем тренировочном датасете
classifier_CB.fit(X_train, y_train)
y_score = classifier_CB.predict_proba(X_test)[:, 1]

Посчитаем precision/recall/f_score

In [70]:
b=1
precision_CB, recall_CB, thresholds_CB = precision_recall_curve(y_test.values, y_score)
fscore_CB = (1+b**2)*(precision_CB * recall_CB) / (b**2*precision_CB + recall_CB)
# locate the index of the largest f score
ix_CB = np.argmax(fscore_CB)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds_CB[ix_CB], 
                                                                        fscore_CB[ix_CB],
                                                                        precision_CB[ix_CB],
                                                                        recall_CB[ix_CB]))

Best Threshold=0.367163, F-Score=0.740, Precision=0.677, Recall=0.814


In [71]:
# посчитаем ROC AUC
roc_auc_CB = roc_auc_score(y_test.values, y_score)
roc_auc_CB

0.8012110424046732

Подберем параметры с помощью кросс-валидации

In [42]:
classifier_CB = Pipeline([
    ('features',feats),
    ('classifier', CatBoostClassifier(verbose = False, random_state = 42)),
])

params={'classifier__learning_rate':[0.1, 0.2, 0.3],
        'classifier__iterations':[500, 700, 900],
        'classifier__depth':[2, 3],
        }

In [43]:
grid = GridSearchCV(classifier_CB,
                    param_grid=params,
                    cv=5,
                    refit=False)

search = grid.fit(X_train, y_train)
search.best_params_

{'classifier__depth': 3,
 'classifier__iterations': 500,
 'classifier__learning_rate': 0.1}

Обучаем модель уже с новыми параметрами

In [72]:
classifier_CB = Pipeline([
    ('features',feats),
    ('classifier',  CatBoostClassifier(iterations=500, depth=3, learning_rate=0.1, random_state=42, verbose = False)),
])
classifier_CB.fit(X_train, y_train)
y_score = classifier_CB.predict_proba(X_test)[:, 1]

In [73]:
b=1
precision_CB, recall_CB, thresholds_CB = precision_recall_curve(y_test.values, y_score)
fscore_CB = (1+b**2)*(precision_CB * recall_CB) / (b**2*precision_CB + recall_CB)
# locate the index of the largest f score
ix_CB = np.argmax(fscore_CB)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds_CB[ix_CB], 
                                                                        fscore_CB[ix_CB],
                                                                        precision_CB[ix_CB],
                                                                        recall_CB[ix_CB]))

Best Threshold=0.386529, F-Score=0.741, Precision=0.691, Recall=0.798


In [74]:
roc_auc_CB = roc_auc_score(y_test.values, y_score)
roc_auc_CB

0.803204332424214

#### 3. вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)

In [75]:
# Запишем метрики в словарь
metrics = {
    'Variant': ['LogisticRegression', 'CatBoost',
    ],
    'precision': [precision_LR[ix_LR], precision_CB[ix_CB]],
    'roc_auc_score': [roc_auc_LR, roc_auc_CB],
    'recall': [recall_LR[ix_LR], recall_CB[ix_CB]],
    'f_score': [fscore_LR[ix_LR], fscore_CB[ix_CB]]
    }  

In [76]:
pd.DataFrame(metrics)

,Variant,precision,roc_auc_score,recall,f_score
0,LogisticRegression,0.64698,0.784257,0.838018,0.730211
1,CatBoost,0.69111,0.803204,0.798041,0.740737


#### 4. сделать выводы о том, какая модель справилась с задачей лучше других

Если оценивать по интегральной метрике f_score, то видим, что CatBoost дает лучшее качество предстказаний

#### 5. (опциональный вопрос) какая метрика (precision_recall_curve или roc_auc_curve) больше подходит в случае сильного дисбаланса классов? (когда объектов одного из классов намного больше чем другого). 

p.s.В вопросе проще разобраться, если вспомнить оси на графике roc auc curve и рассмотреть такой пример:

Имеется 100000 объектов, из которых только 100 - класс "1" (99900 - класс "0", соответственно). 
Допустим, у нас две модели:

- первая помечает 100 объектов как класс 1, но TP = 90
- вторая помечает 1000 объектов как класс 1, но TP такой же - 90

Какая модель лучше и почему? И что позволяет легче сделать вывод - roc_auc_curve или precision_recall_curve?

## Ответ:

In [111]:
# Первая модель
TP = 90
FP = 10
FN = 10
TN = 99900-10
TPR_1 = TP/(TP+FN)
FPR_1 = FP/(FP+TN)
TNR = TN/(FP+TN)
TPR_1, FPR_1, TNR


(0.9, 0.0001001001001001001, 0.9998998998998999)

In [112]:
Precision_1 = TP/(TP + FP)
Precision_1

0.9

In [113]:
Recall_1 = TP/(TP + FN)
Recall_1

0.9

In [114]:
# Вторая модель
TP = 90
FP = 910
FN = 10
TN = 99900-910
TPR_2 = TP/(TP+FN)
FPR_2 = FP/(FP+TN)
TNR = TN/(FP+TN)
TPR_2, FPR_2, TNR

(0.9, 0.00910910910910911, 0.9908908908908909)

In [115]:
abs(FPR_1 - FPR_2)

0.009009009009009009

In [116]:
Precision_2 = TP/(TP + FP)
Precision_2

0.09

In [117]:
Recall_2 = TP/(TP + FN)
Recall_2

0.9

In [118]:
Precision_1 - Precision_2

0.81

Кривая  roc_auc_curve строится в координатах TPR и FPR. А из расчетов мы видим, что между TPR_1 и TPR_2 нет разности вообще (это полнота), а между FPR_1 и FPR_2 разница есть, но она меньше 1%. Поэтому при таком большом дисбалансе классов roc_auc_curve даст примерно одинаковые оценки моделям.  
Кривая precision_recall_curve строится в координатах Precision и Recall. И если Recall, как я уже говорил, у моделей одинаковый, то вот Precision у первой модели выше на 81%. Таким образом побеждает 1 модель и такой  вывод помогает сделать именно precision_recall_curve, которая покажет большую разницу между моделями, чем roc_auc_curve.

# <font color='red'> Сообщение ревьюеру: <font color='black'>
1. MinMaxScaler пробовал, результат хуже
2. Вы мне несколько раз указывали на отсутвие кросс-валидации. Если я применяю GridSearchCV, это же считается за кросс-валидации